In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
path = Path('/home/jupyter/fpl-prediction/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
76709,Yoshinori_Muto,16,4,0,Newcastle United,Southampton,0.561705,0.469252,True,0,1920,1.0
76710,Youri_Tielemans,16,3,90,Leicester City,Aston Villa,1.054253,0.522434,False,2,1920,1.0
76711,Yves_Bissouma,16,3,0,Brighton and Hove Albion,Wolverhampton Wanderers,0.453089,0.746215,True,0,1920,1.0
76712,Çaglar_Söyüncü,16,2,90,Leicester City,Aston Villa,1.054253,0.522434,False,2,1920,1.0
76713,Ørjan_Nyland,16,1,0,Aston Villa,Leicester City,0.522434,1.054253,True,0,1920,1.0


In [5]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

17

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [7]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
0,Brandon_Austin,17,1,0,Tottenham Hotspur,Wolverhampton Wanderers,2.001099,0.746215,False,0,1920
1,Tom_Davies,17,3,90,Everton,Manchester United,1.033814,1.560524,False,1,1920
2,Lucas_Torreira,17,3,80,Arsenal,Manchester City,1.413491,2.711234,True,2,1920
3,Danny_Ings,17,4,90,Southampton,West Ham United,0.469252,0.699185,True,2,1920
4,Ibrahim_Amadou,17,3,0,Norwich,Leicester City,0.295212,1.054253,False,0,1920
5,Willy_Caballero,17,1,0,Chelsea,Bournemouth,1.757609,0.713784,True,0,1920
6,Kevin_Long,17,2,0,Burnley,Newcastle United,0.406164,0.561705,True,0,1920
7,Harry_Lewis,17,1,0,Southampton,West Ham United,0.469252,0.699185,True,0,1920
8,Adam_Smith,17,2,0,Bournemouth,Chelsea,0.713784,1.757609,False,0,1920
9,Karl_Darlow,17,1,0,Newcastle United,Burnley,0.561705,0.406164,False,0,1920


In [8]:
df_latest.shape

(576, 11)

In [9]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [10]:
last_play_proba.head()

,player,play_proba
0,Shkodran_Mustafi,1.00
1,Héctor_Bellerín,0.75
2,Sead_Kolasinac,1.00
3,Ainsley_Maitland-Niles,1.00
4,Sokratis_Papastathopoulos,1.00


In [11]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [12]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
0,Brandon_Austin,17,1,0,Tottenham Hotspur,Wolverhampton Wanderers,2.001099,0.746215,False,0,1920,1.00
1,Tom_Davies,17,3,90,Everton,Manchester United,1.033814,1.560524,False,1,1920,1.00
2,Lucas_Torreira,17,3,80,Arsenal,Manchester City,1.413491,2.711234,True,2,1920,1.00
3,Danny_Ings,17,4,90,Southampton,West Ham United,0.469252,0.699185,True,2,1920,1.00
4,Ibrahim_Amadou,17,3,0,Norwich,Leicester City,0.295212,1.054253,False,0,1920,1.00
5,Willy_Caballero,17,1,0,Chelsea,Bournemouth,1.757609,0.713784,True,0,1920,1.00
6,Kevin_Long,17,2,0,Burnley,Newcastle United,0.406164,0.561705,True,0,1920,0.50
7,Harry_Lewis,17,1,0,Southampton,West Ham United,0.469252,0.699185,True,0,1920,1.00
8,Adam_Smith,17,2,0,Bournemouth,Chelsea,0.713784,1.757609,False,0,1920,0.00
9,Karl_Darlow,17,1,0,Newcastle United,Burnley,0.561705,0.406164,False,0,1920,1.00


In [13]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [14]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
77285,Wilfried_Zaha,17,3,90,Crystal Palace,Brighton and Hove Albion,0.464142,0.453089,True,8,1920,1.0
77286,Michael_Keane,17,2,90,Everton,Manchester United,1.033814,1.560524,False,2,1920,1.0
77287,Jairo_Riedewald,17,3,45,Crystal Palace,Brighton and Hove Albion,0.464142,0.453089,True,1,1920,1.0
77288,Aaron_Mooy,17,3,90,Brighton and Hove Albion,Crystal Palace,0.453089,0.464142,False,2,1920,1.0
77289,John_Ruddy,17,1,0,Wolverhampton Wanderers,Tottenham Hotspur,0.746215,2.001099,True,0,1920,1.0


In [15]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [16]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [17]:
fixtures.head(10)

,home_team,away_team,gw
170,Aston Villa,Southampton,18
171,Bournemouth,Burnley,18
172,Brighton and Hove Albion,Sheffield United,18
173,Everton,Arsenal,18
174,Manchester City,Leicester City,18
175,Newcastle United,Crystal Palace,18
176,Norwich,Wolverhampton Wanderers,18
177,Tottenham Hotspur,Chelsea,18
178,Watford,Manchester United,18
179,West Ham United,Liverpool,34


In [18]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [19]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,18,Aston Villa,Southampton,7,20
1,18,Bournemouth,Burnley,91,90
2,18,Brighton and Hove Albion,Sheffield United,36,49
3,18,Everton,Arsenal,11,3
4,18,Manchester City,Leicester City,43,13


In [20]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [21]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [22]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,18,Southampton,Aston Villa,Jannik_Vestergaard,2,48,None,False
1,18,Southampton,Aston Villa,Ryan_Bertrand,2,48,100,False
2,18,Southampton,Aston Villa,Yan_Valery,2,46,0,False
3,18,Southampton,Aston Villa,Jan_Bednarek,2,44,None,False
4,18,Southampton,Aston Villa,Jack_Stephens,2,43,None,False


In [23]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,18,Aston Villa,Southampton,Ahmed_El Mohamady,2,42,None,True
1,18,Aston Villa,Southampton,James_Chester,2,43,100,True
2,18,Aston Villa,Southampton,Neil_Taylor,2,44,None,True
3,18,Aston Villa,Southampton,Kortney_Hause,2,44,100,True
4,18,Aston Villa,Southampton,Jonathan_Kodjia,4,54,100,True


In [24]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 25.0,
 'Aaron_Cresswell': 80.0,
 'Aaron_Lennon': 24.4,
 'Aaron_Mooy': 90.0,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abdoulaye_Doucouré': 86.6,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 23.0,
 'Adam_Masina': 63.0,
 'Adam_Smith': 45.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 89.5,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 72.0,
 'Adrián_San Miguel del Castillo': 20.4,
 'Ahmed_El Mohamady': 36.0,
 'Ainsley_Maitland-Niles': 36.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 2.0,
 'Alex_Iwobi': 76.2,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 49.2,
 'Alexander_Tettey': 67.5,
 'Alexandre_Lacazette': 51.2,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 4.4,
 'Alisson_Ramses Becker': 86.25,
 'Allan_Saint-Maximin': 84.5,
 'Andre_Gray': 21.8,
 'Andreas_Christensen': 36.0,
 'Andreas_Pereira': 37.6,
 'Andrew_Robertson': 75.2,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 25.5,
 'Andros_Townsend': 55.0,
 'Andy_Carroll': 37.0,
 'Andy_Lonergan': 0.0,

In [25]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [26]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,18,Aston Villa,Southampton,Ahmed_El Mohamady,2,4.2,1.00,True,0.522434,0.469252,1920,36.000000
1,18,Aston Villa,Southampton,James_Chester,2,4.3,1.00,True,0.522434,0.469252,1920,0.000000
2,18,Aston Villa,Southampton,Neil_Taylor,2,4.4,1.00,True,0.522434,0.469252,1920,0.000000
3,18,Aston Villa,Southampton,Kortney_Hause,2,4.4,1.00,True,0.522434,0.469252,1920,18.000000
4,18,Aston Villa,Southampton,Jonathan_Kodjia,4,5.4,1.00,True,0.522434,0.469252,1920,4.200000
5,18,Aston Villa,Southampton,Ørjan_Nyland,1,4.3,1.00,True,0.522434,0.469252,1920,0.000000
6,18,Aston Villa,Southampton,Conor_Hourihane,3,5.7,1.00,True,0.522434,0.469252,1920,45.400000
7,18,Aston Villa,Southampton,Jack_Grealish,3,6.2,1.00,True,0.522434,0.469252,1920,90.000000
8,18,Aston Villa,Southampton,Anwar_El Ghazi,3,5.5,1.00,True,0.522434,0.469252,1920,62.000000
9,18,Aston Villa,Southampton,John_McGinn,3,5.7,1.00,True,0.522434,0.469252,1920,90.000000


In [27]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')